In [1]:
import boto3
import pandas as pd
import numpy as np
from boto3 import client
import os
# Let's use Amazon S3
s3 = boto3.resource('s3')

In [2]:
buckets_check = []
# Print out bucket names
for bucket in s3.buckets.all():
    name = bucket.name
    if name not in ['log.maymobility.com','japan-logs-maymobility','data-pipeline.maymobility.com','log.japan.maymobility.com']:
        buckets_check.append(name)

In [3]:
buckets_check = pd.DataFrame(buckets_check,columns = ['bucket_name'])

In [4]:
def get_bucket_size(bucket_name):
    size = {}
    try:
        for object in boto3.resource('s3').Bucket(bucket_name).objects.all():
            if object.storage_class in size:
                size[object.storage_class] += object.size
            else:
                size[object.storage_class] = object.size
        print(bucket_name, size)
        return size
    except:
        print(bucket_name,'not working')
        return 'none'
# storage class


In [5]:
buckets_check['bucket_size'] = buckets_check.apply(lambda x: get_bucket_size(x.bucket_name),axis = 1)

airflow-data-maymobility.com {}
amazon-connect-b275969a4e94 {}
annotations.simdata.maymobility.com {'STANDARD': 1211252819282}
applied.maymobility.com {'STANDARD': 503375161632}
artifacts.code.maymobility.com {'STANDARD': 16887411630}
assetprotection.maymobility.com {'STANDARD': 686524497}
audit-log.maymobility.com not working
auto-snipping {'STANDARD': 9957771147286}
aws-simplewebstreaming-hostingbucket-783236726606-dev {'STANDARD': 12645473}
backup.maymobility.com {'GLACIER': 73187852300927}
cdn.code.maymobility.com {}
cf-templates-18g0p5n6agm8r-us-east-1 {'STANDARD': 21733}
cloudtrail-logs-maymobility {'STANDARD': 459017}
compressed-logs-may {}
config-bucket-783236726606 {'STANDARD': 9205614}
crl.its.maymobility.com {'STANDARD': 1090}
dashboard.maymobility.com {}
dependency-proxy.code.maymobility.com {}
docs.api.maymobility.com {'STANDARD': 21459}
eks-quickstart-lambdazips-us-east-2-783236726606 {'STANDARD': 333161777}
external-diffs.code.maymobility.com {}
gitlab-runner-cache.maymo

In [47]:
buckets_check[buckets_check.bucket_size == 'none']
# my access to audit-log.maymobility.com is denied and I couldn't get its size

,bucket_name,bucket_size
6,audit-log.maymobility.com,none


In [44]:
data = buckets_check[buckets_check.bucket_size != 'none'].bucket_size.to_list()
result = pd.DataFrame.from_records(data,index = buckets_check[buckets_check.bucket_size != 'none'].bucket_name)

In [38]:
result = result.fillna(0)
result = result.astype(int)
result['total_size'] = result.STANDARD + result.GLACIER + result.INTELLIGENT_TIERING + result.DEEP_ARCHIVE
# size unit is byte

In [46]:
result = result.sort_values(by = 'total_size',ascending = False)

In [48]:
result.to_csv('size_result.csv')